In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient

In [3]:
PI_table1 = pd.DataFrame(Graph_Entropy_measure.PI_Rank, columns = ('nodes_PI','PI_i'))
#PI_table1.PI_i[PI_table.PI_i  < 0] = 0
PI_table = PI_table1.copy()
i = math.sqrt(2)
PI_table.PI_i = PI_table.PI_i.where(lambda x: x > 0, lambda x: x + i)

In [4]:
TC_table = pd.DataFrame(Tr_centrality_measure.TC_Rank, columns = ('nodes_TC','TC_i'))
#TC_table.nodes_TC = TC_table.nodes_TC + 1
CC_table = pd.DataFrame(Constraint_Coefficient.sorted_x, columns = ('nodes_CC','CC_i'))
#CC_table.nodes_CC= CC_table.nodes_CC + 1
Result_table = pd.concat([TC_table, PI_table, CC_table], axis=1, sort=False)

First_Five= pd.DataFrame(Result_table.loc[0:2])
First_Five.index = First_Five.index + 1
First_Five

,nodes_TC,TC_i,nodes_PI,PI_i,nodes_CC,CC_i
1,4326,0.4242,4326,0.328181,2681,0.097191
2,4362,0.4219,2543,0.335492,2582,0.121122
3,4332,0.4138,4316,0.346705,1066,0.121128


In [5]:
#return the total power of node i.
def TPN(i):
    PI_ofi = 1/Graph_Entropy_measure.PI_of(i)
    TC_ofi = Tr_centrality_measure.TC_of(i)
    CC_ofi = Constraint_Coefficient.CC_of(i)
    #Weight to normalize the measures
    dn=PI_ofi+TC_ofi+CC_ofi
    n1=round(TC_ofi/dn, 2)
    n2=round(PI_ofi/dn, 2)
    n3=round(CC_ofi/dn, 2)
    #equation []
    TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)       
    return TPN_i

In [6]:
#loop total power of node all i
def loop_TPN():
    i=0
    nodes = list(pd.concat([First_Five['nodes_TC'], First_Five['nodes_PI'], First_Five['nodes_CC']]).unique())
    list_TPN=[]    
    for i in nodes:          
        TPN_ = TPN(i)
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)
    return  list_TPN

In [7]:
#Ranking the total power of node i
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1], reverse=True)

In [8]:
#return table containing the TPN values[TPN_table and Top_Five]
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes_TPN','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:4])
Top_Five.index = Top_Five.index + 1
#TPN_table.nodes_TPN = TPN_table.nodes_TPN + 1
print(TPN_table)
#TPN_table.to_csv('result/TPN_[]_network.csv')
#Top_Five.to_csv('result/TPN_[]_network.csv')

   nodes_TPN     TPN_i
0       1066  0.521788
1       4332  0.369001
2       4362  0.364213
3       2582  0.352836
4       2681  0.344294
5       4326  0.341308
6       4316  0.335686
7       2543  0.330789


In [9]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_Five = pd.DataFrame(Propose_Result_table.loc[0:2])
Top_Five.index = Top_Five.index + 1
#convert column nodes_TPN to int
#Top_Five.nodes_TPN= Top_Five.nodes_TPN.astype(int)
print(Top_Five)
#Propose_Table.to_csv('result/Propose_Table_kc_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC    TC_i  nodes_PI      PI_i  nodes_CC      CC_i  nodes_TPN  \
1      4326  0.4242      4326  0.328181      2681  0.097191     1066.0   
2      4362  0.4219      2543  0.335492      2582  0.121122     4332.0   
3      4332  0.4138      4316  0.346705      1066  0.121128     4362.0   

      TPN_i  
1  0.521788  
2  0.369001  
3  0.364213  


In [10]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_FiveR = pd.DataFrame(Propose_Result_table.loc[0:2])
Top_FiveR.index = Top_FiveR.index + 1
#convert column nodes_TPN to int
Top_FiveR.nodes_TPN= Top_FiveR.nodes_TPN.astype(int)
ranking_table = pd.concat([Top_FiveR['nodes_TC'], Top_FiveR['nodes_PI'], Top_FiveR['nodes_CC'],Top_FiveR['nodes_TPN']], axis=1, sort=False)
print(ranking_table)
#Propose_Table.to_csv('result/Propose_Table_power_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC  nodes_PI  nodes_CC  nodes_TPN
1      4326      4326      2681       1066
2      4362      2543      2582       4332
3      4332      4316      1066       4362


In [11]:
# calculate the kendall's correlation between two variables# calcul 
from numpy.random import rand
from numpy.random import seed
from scipy.stats import kendalltau

# prepare data
data1 = Top_FiveR['TPN_i']
data2 = data1 + Top_FiveR['CC_i']+ Top_FiveR['PI_i']+ Top_FiveR['TC_i']
# calculate kendall's correlation
coef, p = kendalltau(data1, data2)
print('Kendall correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Kendall correlation coefficient: 1.000
Samples are uncorrelated (fail to reject H0) p=0.117


In [40]:
cent = pd.read_csv('result/power/Top_Five_cent_Power_network.csv')
all_in=pd.concat([cent,Propose_Result_table], axis=1, sort=False)
all_in.nodes_PI = all_in.nodes_PI + 1
all_in.nodes_TC = all_in.nodes_TC + 1
all_in.nodes_CC = all_in.nodes_CC + 1
all_in.nodes_TPN = all_in.nodes_TPN + 1
Top_Five_all_in = pd.DataFrame(all_in.loc[0:2])
ranking_table_all_in = pd.concat([Top_Five_all_in['nodes_tr'], Top_Five_all_in['nodes_bc'], Top_Five_all_in['nodes_cnc'],
                                  Top_Five_all_in['nodes_dc'], Top_Five_all_in['nodes_ec'],Top_Five_all_in['nodes_pr'],
                                  Top_Five_all_in['nodes_TC'], 
                                  Top_Five_all_in['nodes_PI'], Top_Five_all_in['nodes_CC'],Top_Five_all_in['nodes_TPN']], 
                                 axis=1, sort=False)

#ranking_table_all_in.to_csv('result/ranking_table_all_in_power.csv')
ranking_table_all_in

,nodes_tr,nodes_bc,nodes_cnc,nodes_dc,nodes_ec,nodes_pr,nodes_TC,nodes_PI,nodes_CC,nodes_TPN
0,4363.0,4148.0,1268.0,2682.0,4361.0,818.0,4327,4327,2682,1067.0
1,4327.0,2504.0,2554.0,4447.0,4327.0,4447.0,4363,2544,2583,4333.0
2,4333.0,1200.0,2570.0,818.0,4319.0,3413.0,4333,4317,1067,4363.0


In [37]:
# Importing required libraries
import researchpy as rp
from scipy import stats
#The casewise method returns all information in 3 dataframes
#corr_type, corr_matrix, corr_ps=>
corr_type, corr_matrix, corr_ps = rp.corr_case(Top_Five_all_in[['TR', 'BC', 'CnC', 'DC', 'EC','pr','TC_i', 'PI_i', 'CC_i', 'TPN_i']])
#corr_type, corr_matrix, corr_ps = rp.corr_case(ranking_table_all_in[['nodes_tr', 'nodes_bc', 'nodes_cnc', 'nodes_dc', 'nodes_ec','nodes_pr','nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])
#PAIRWISEcorrelation
corr_pair = rp.corr_pair(ranking_table_all_in[['nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])

In [38]:
corr_matrix
#corr_matrix.to_csv('result/corr_matrix_power.csv')

,TR,BC,CnC,DC,EC,pr,TC_i,PI_i,CC_i,TPN_i
TR,1,0.9546,0.9934,0.9449,0.9776,0.9909,0.9519,-0.9927,-0.8661,0.8791
BC,0.9546,1,0.9825,0.8045,0.8706,0.9057,0.8173,-0.9116,-0.9757,0.9812
CnC,0.9934,0.9825,1,0.901,0.947,0.9688,0.9104,-0.9722,-0.9178,0.928
DC,0.9449,0.8045,0.901,1,0.9926,0.9804,0.9998,-0.9775,-0.6548,0.6746
EC,0.9776,0.8706,0.947,0.9926,1,0.9971,0.995,-0.9959,-0.7417,0.7592
pr,0.9909,0.9057,0.9688,0.9804,0.9971,1,0.9845,-0.9999,-0.7908,0.8068
TC_i,0.9519,0.8173,0.9104,0.9998,0.995,0.9845,1,-0.9819,-0.6713,0.6907
PI_i,-0.9927,-0.9116,-0.9722,-0.9775,-0.9959,-0.9999,-0.9819,1,0.7995,-0.8151
CC_i,-0.8661,-0.9757,-0.9178,-0.6548,-0.7417,-0.7908,-0.6713,0.7995,1,-0.9996
TPN_i,0.8791,0.9812,0.928,0.6746,0.7592,0.8068,0.6907,-0.8151,-0.9996,1


In [15]:
#Graph_Entropy_measure.PI_of(23)